### Example of TSSOS failure to find solution on 4-th degree polynomial

Consider simplest possible Lindblad master equation with just one dissipator:

$
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right]
$,

where Hamiltonian is 


$
    H_0 = |1\rangle \langle 1| = \begin{pmatrix} h & 0 \\ 0 & 0
   \end{pmatrix}
$

$
A = \sqrt{\gamma} \sigma = \begin{pmatrix} 0 & \sqrt{\gamma} \\ 0 & 0
   \end{pmatrix} = \begin{pmatrix} 0 & a \\ 0 & 0
   \end{pmatrix}$

$\sigma = |0\rangle \langle 1|  = \begin{pmatrix} 0 & 1 \\ 0 & 0
   \end{pmatrix}$


In [1]:
using DynamicPolynomials

@polyvar E

    Hˢʸᵐᵇ = [ E   0
              0   0. ]

@polyvar a

    Aˢʸᵐᵇ = [ 0  a
              0  0.]

2×2 Matrix{Term{true, Float64}}:
 0.0  a
 0.0  0.0

Load exact data and essemble objective

In [2]:
include("LiPoSID.jl")

Main.LiPoSID

In [3]:
parentdir = pwd()

γᵗˣᵗ = "0.079477"

datadir = parentdir*"\\DATA\\"

ρᵍ, tᵍ = LiPoSID.get_rho_series(datadir*"State_B1_2CUT_data.h5", γᵗˣᵗ)
ρᵉ, tᵉ = LiPoSID.get_rho_series(datadir*"State_B2_2CUT_data.h5", γᵗˣᵗ)
ρˣ, tˣ = LiPoSID.get_rho_series(datadir*"State_B3_2CUT_data.h5", γᵗˣᵗ)
ρʸ, tʸ = LiPoSID.get_rho_series(datadir*"State_B4_2CUT_data.h5", γᵗˣᵗ)

ρˣ = convert(Vector{Matrix{ComplexF64}}, ρˣ)
ρʸ = convert(Vector{Matrix{ComplexF64}}, ρʸ)
ρᵍ = convert(Vector{Matrix{ComplexF64}}, ρᵍ)
ρᵉ = convert(Vector{Matrix{ComplexF64}}, ρᵉ)
t = convert(Vector{Float64}, tˣ);

obj = LiPoSID.simpson_obj(ρᵉ, t, Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
obj += LiPoSID.simpson_obj(ρᵍ, t, Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
obj += LiPoSID.simpson_obj(ρˣ, t, Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
obj += LiPoSID.simpson_obj(ρʸ, t, Hˢʸᵐᵇ, [Aˢʸᵐᵇ])

1.6697789826640932a⁴ + 0.7949238686720486E² - 0.26545473091539a² - 39.94662884489179E + 501.8627949039175

Here we have polynomial of degree 4 which is symmetric in valiable $a$. 

It looks like it is somehow patalogic for TSSOS :(

In [4]:
using TSSOS

In [7]:
opt,sol,data = tssos_first(obj, variables(obj), QUIET=true, solution=true);
previous_sol = sol

************************TSSOS************************
TSSOS is launching...
optimum = 0.0013068260669271202
Global optimality certified!


2-element Vector{Float64}:
 25.126072080100073
  0.28193614217535945

In [6]:
opt,sol,data = tssos_first(obj, variables(obj), QUIET=true, solution=true, newton=false);
previous_sol = sol

************************TSSOS************************
TSSOS is launching...
optimum = 0.0013068252977344087
The local solver failed!


2-element Vector{Float64}:
 25.126084864809815
  1.0e-10

In [6]:
opt, sol, data = tssos_higher!(data; QUIET=true, solution=true);

optimum = 0.001306828394690757
Found a local optimal solution giving an upper bound: 0.011857047039541158 and a relative optimality gap: 0.0105502186448504.


In [7]:
sol

2-element Vector{Float64}:
 25.12607207470193
  9.807635928052407e-10

In [8]:
solution = variables(obj) => sol

PolyVar{true}[E, a] => [25.12607207470193, 9.807635928052407e-10]

In [9]:
subs(Hˢʸᵐᵇ, solution)

2×2 Matrix{Term{true, Float64}}:
 25.1261  0.0
 0.0      0.0

In [10]:
subs(Aˢʸᵐᵇ, solution)

2×2 Matrix{Term{true, Float64}}:
 0.0  9.80764e-10
 0.0  0.0

The dissipator is almoust zero which is simply not true!!!

### Reformulate the same problem 

Lindblad master equation could be rewritten in even simpler form if put $\gamma$ out of brackets:

$
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right]
$

$
A = \sqrt{\gamma} \sigma = \begin{pmatrix} 0 & \sqrt{\gamma} \\ 0 & 0
   \end{pmatrix} = \begin{pmatrix} 0 & a \\ 0 & 0
   \end{pmatrix}$
   
$
    \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+ \gamma \left[\sigma \rho \sigma^\dagger - \frac{1}{2} \left\{ \sigma^\dagger \sigma, \rho \right\} \right]
$


$\sigma = |0\rangle \langle 1|  = \begin{pmatrix} 0 & 1 \\ 0 & 0
   \end{pmatrix}$

In [30]:
function γ_rhs(ρ, H, γ)
    """
    Right hand side of the Lindblad master equation
    """
    
    𝜎 = [ 0  1
          0  0. + 0im ]
    
    return -im * (H * ρ - ρ * H) + γ * (𝜎 * ρ * 𝜎' - (𝜎' * 𝜎  * ρ + ρ * 𝜎' * 𝜎) / 2)
    
end

function γ_simpsom_obj(ρ::Vector{Matrix{ComplexF64}}, t, H, γ)
    
    obj = 0
    for i in 3:length(ρ)
        obj += LiPoSID.frobenius_norm2(
            ρ[i] - ρ[i-2] - (t[i]-t[i-1])γ_rhs((ρ[i-2] + 4ρ[i-1] + ρ[i])/3, H, γ)
        )
    end
    obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    return obj
end

γ_simpsom_obj (generic function with 1 method)

In [31]:
γᶠˡᵒᵃᵗ = parse(Float64, γᵗˣᵗ)

0.079477

In [32]:
𝜎 = [ 0  1
      0  0. + 0im ]

obj_γ = γ_simpsom_obj(ρᵉ, t, Hˢʸᵐᵇ, γᶠˡᵒᵃᵗ)
obj_γ += γ_simpsom_obj(ρᵍ, t, Hˢʸᵐᵇ, γᶠˡᵒᵃᵗ)
obj_γ += γ_simpsom_obj(ρˣ, t, Hˢʸᵐᵇ, γᶠˡᵒᵃᵗ)
obj_γ += γ_simpsom_obj(ρʸ, t, Hˢʸᵐᵇ, γᶠˡᵒᵃᵗ)

0.7949238686720486E² - 39.94662884489179E + 501.85224467338554

which is of course very simple quadratic polynomial

In [33]:
opt_γ, sol_γ, data_γ = tssos_first(obj_γ, variables(obj_γ), QUIET=true, solution=true, newton=false);|

************************TSSOS************************
TSSOS is launching...
optimum = 0.0013068165323672483
Global optimality certified!


| (generic function with 36 methods)

The Hamiltonian remains almoust the same:

In [34]:
solution = variables(obj_γ) => sol_γ
subs(Hˢʸᵐᵇ, solution)

2×2 Matrix{Term{true, Float64}}:
 25.1261  0.0
 0.0      0.0

but we can find another dissipator $A$ that can fit the data much better

In [35]:
√γᶠˡᵒᵃᵗ*𝜎

2×2 Matrix{ComplexF64}:
 0.0+0.0im  0.281917+0.0im
 0.0+0.0im       0.0+0.0im

Thus for some reason TSSOS fails to find the right solution :(

#### We can also solve it as 2D optimization problem in $E$ and $\gamma$

In [41]:
@polyvar γ

(γ,)

In [42]:
obj_γˢʸᵐᵇ = γ_simpsom_obj(ρᵉ, t, Hˢʸᵐᵇ, γ)
obj_γˢʸᵐᵇ += γ_simpsom_obj(ρᵍ, t, Hˢʸᵐᵇ, γ)
obj_γˢʸᵐᵇ += γ_simpsom_obj(ρˣ, t, Hˢʸᵐᵇ, γ)
obj_γˢʸᵐᵇ += γ_simpsom_obj(ρʸ, t, Hˢʸᵐᵇ, γ)

0.7949238686720486E² + 1.6697789826640932γ² - 39.94662884489179E - 0.26545473091539γ + 501.8627949039175

In [44]:
opt_γˢʸᵐᵇ, sol_γˢʸᵐᵇ, data_γˢʸᵐᵇ = tssos_first(obj_γˢʸᵐᵇ, variables(obj_γˢʸᵐᵇ), QUIET=true, solution=true, newton=false);|

************************TSSOS************************
TSSOS is launching...
optimum = 0.001306817252991087
Global optimality certified!


| (generic function with 36 methods)

In [47]:
solution = variables(obj_γˢʸᵐᵇ) => sol_γˢʸᵐᵇ
subs(Hˢʸᵐᵇ, solution)

2×2 Matrix{Term{true, Float64}}:
 25.1261  0.0
 0.0      0.0

In [48]:
subs(γ, solution)

0.07948797559101574

Almoust a perfect fit to the analitical ansatz

In [9]:
0.28193614217535945^2

0.0794879882647245